O código presente neste arquivo implementa uma solução para capturar imagens de satélite a partir de coordenadas de latitude e longitude. As imagens obtidas serão integradas ao projeto original de rede neural, contribuindo para a análise e predição de fraudes. Os resultados gerados neste Colab servirão como insumo para o desenvolvimento e aprimoramento do modelo de rede neura

##Importando os dados

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Substitua pelo caminho correto para o seu arquivo
file_path = '/content/drive/Shareddrives/M11_SI/MERGE_ROBUST.parquet'
df = pd.read_parquet(file_path)


In [ ]:
df_lat_long = df[['MATRICULA','COD_LATITUDE', 'COD_LONGITUDE']]


In [ ]:
df_unique = df_lat_long.drop_duplicates(subset='MATRICULA', keep='first')


##Convertendo lat e long para TILE

In [ ]:
import requests
from io import BytesIO
from PIL import Image
import math
import pandas as pd

# URL da XYZ Tile do Google Earth
TILE_URL = "https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}"

# Função para converter latitude e longitude em coordenadas de tile
def latlon_to_tile(lat, lon, zoom):
    lat_rad = math.radians(lat)
    n = 2.0 ** zoom
    x_tile = int((lon + 180.0) / 360.0 * n)
    y_tile = int((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi) / 2.0 * n)
    return x_tile, y_tile


##Dividindo o DF

In [ ]:
import numpy as np


# Dividir o DataFrame em 3 partes
df_split = np.array_split(df_unique, 5)

# Acesso às partes divididas
df_part2 = df_split[4]


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


##Função de busca de imagem

In [ ]:
def download_image(lat, lon, zoom=18):
    TILE_URL = "https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}"
    x_tile, y_tile = latlon_to_tile(lat, lon, zoom)
    url = TILE_URL.format(x=x_tile, y=y_tile, z=zoom)
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

##Convertendo as imagens em features

In [ ]:
import gc
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
import numpy as np
import tensorflow as tf

# Limitar o uso de memória da GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Carrega o modelo VGG16 com pooling global (uso médio)
model = VGG16(weights='imagenet', include_top=False, pooling='avg')

# Inicializa a coluna 'features' com valores vazios
df_part2['features'] = None

# Define o tamanho do batch
batch_size = 32
batch_imgs = []
batch_indices = []

for i, (_, row) in enumerate(df_part2.iterrows()):
    try:
        print(f"Processando matrícula {row['MATRICULA']}...")
        img = download_image(row['COD_LATITUDE'], row['COD_LONGITUDE'])

        if img is not None:
            img = img.resize((224, 224))
            img_array = image.img_to_array(img)
            img_array = preprocess_input(img_array)
            batch_imgs.append(img_array)
            batch_indices.append(row.name)

            # Processa o batch quando atinge o tamanho definido
            if len(batch_imgs) == batch_size:
                batch_imgs_array = np.array(batch_imgs)
                features = model.predict(batch_imgs_array)

                for idx, feature in zip(batch_indices, features):
                    df_part2.at[idx, 'features'] = feature.flatten().tolist()

                # Limpa o batch
                batch_imgs = []
                batch_indices = []

                # Chama o garbage collector para liberar memória
                gc.collect()

            img.close()
        else:
            print(f"Imagem não encontrada para a matrícula {row['MATRICULA']}")

    except Exception as e:
        print(f"Erro ao processar a matrícula {row['MATRICULA']}: {e}")

# Processa o restante das imagens do batch que não foi completo
if batch_imgs:
    batch_imgs_array = np.array(batch_imgs)
    features = model.predict(batch_imgs_array)
    for idx, feature in zip(batch_indices, features):
        df_part2.at[idx, 'features'] = feature.flatten().tolist()

    # Limpeza final
    gc.collect()

# Verifica os resultados na coluna 'features'
df_part2[['MATRICULA', 'features']]


A saída de streaming foi truncada nas últimas 5000 linhas.
Processando matrícula 17973913...
Processando matrícula 17973317...
Processando matrícula 17967824...
Processando matrícula 17966685...
Processando matrícula 17970444...
Processando matrícula 17968408...
Processando matrícula 17970005...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Processando matrícula 17966213...
Processando matrícula 17965327...
Processando matrícula 17982714...
Processando matrícula 17976031...
Processando matrícula 17974356...
Processando matrícula 17975920...
Processando matrícula 17973332...
Processando matrícula 17973168...
Processando matrícula 17976660...
Processando matrícula 17947846...
Processando matrícula 17972267...
Processando matrícula 17976497...
Processando matrícula 17973964...
Processando matrícula 17962694...
Processando matrícula 17975641...
Processando matrícula 17974285...
Processando matrícula 17970988...
Processando matrícula 17979710...
Processando matrícula 17951480...
Processando matrícu

,MATRICULA,features
1308805,17424754,"[0.2168809175491333, 0.0, 0.7251922488212585, ..."
1308807,17456344,"[0.14695625007152557, 0.0, 1.4281771183013916,..."
1308815,17424757,"[0.2168809175491333, 0.0, 0.7251922488212585, ..."
1308825,17549825,"[2.241490602493286, 0.0, 2.0740511417388916, 0..."
1308828,17786031,"[0.3767499625682831, 0.0, 0.0, 0.0, 0.30879673..."
...,...,...
7590908,17968968,"[0.45997142791748047, 0.24972467124462128, 1.4..."
7593991,17982908,"[0.8476272225379944, 0.0, 0.13513189554214478,..."
7594057,17978800,"[1.748569369316101, 0.3191480040550232, 0.0, 0..."
7594060,17981661,"[1.451476812362671, 0.0, 2.3614978790283203, 0..."


##Juntando as bases e exportando

In [ ]:
# Aplica a função somente se o valor não for None
df_part2['features'] = df_part2['features'].apply(lambda x: ','.join(map(str, x)) if x is not None else '')


In [ ]:
# Exporta para CSV
df_part2.to_csv('df5.csv', index=False)

In [ ]:
df1 = pd.read_csv('/content/df1.csv')
df2 = pd.read_csv('/content/df2.csv')
df3 = pd.read_csv('/content/df30.csv')
df4 = pd.read_csv('/content/df4.csv')
df5 = pd.read_csv('/content/df5.csv')

# Usando o pd.concat para juntar os DataFrames
df_final = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

In [ ]:
df_final.to_csv('baseImagem.csv', encoding='utf-8', index=False)